In [ ]:
from csv import DictReader

from astropy.coordinates import SkyCoord
from astropy import units as u

from pocs.core import POCS

### Read from CSV

In [ ]:
positions_fn = 'accel-positions.csv'

In [ ]:
# Show top of file
!head -n 3 $positions_fn

In [ ]:
# For each row, create a SkyCoord

coords = list()

with open(positions_fn, 'r') as csv_file:
    reader = DictReader(csv_file)
    for row in reader:
        coords.append(SkyCoord(**row)) # The ** passes the keys of the dict as params

### Create POCS instance

Here we create an instance of POCS that uses the simulator. You can do just `pocs = POCS()` to control a real connected mount.

In [ ]:
pocs = POCS(simulator=['all']) 

In [ ]:
pocs.initialize()

In [ ]:
mount = pocs.observatory.mount

In [ ]:
mount.unpark()
mount.slew_to_home()

In [ ]:
# Set the delay to wait at each position. Probably should be something
# like 30 seconds to ensure the sensor readings have "caught up" to the
# mount movement.

delay = 1 # seconds

In [ ]:
for coord in coords:
    print("Slewing to {}".format(coord))
    mount.set_target_coordinates(coord)
    mount.slew_to_target()
    
    print('Collecting stats for {} seconds'.format(delay))
    pocs.sleep(delay)

In [ ]:
mount.home_and_park()